In [31]:
import os
import pandas as pd
import numpy as np

In [32]:


def rename_file_with_updirectory(address, search_string):
    '''
    #searches files containing provided substring and included updirectory name in th filename

    #Parameters:
        address : string, address of the location of the files
        search_string: string, search object

    #Returns:
        None 
    '''
    for root, subdirs, files in os.walk(address):
        for name in files:
            if search_string in name:
                curr_address = os.path.basename(root)
                oldname = os.path.join(address, curr_address, name)
                splt_name =  name.split('.')
                myname = '_'.join([curr_address,splt_name[0] + '.' + splt_name[1]]) ##name arangement can be changed here
                newname = os.path.join(address, curr_address, myname)
                os.rename(oldname, newname)

In [33]:
# rename_file_with_updirectory(os.getcwd(),'.xlsx')

In [34]:

def list_files_containing_search_string(address = os.getcwd(), search_string ='.' ):
    '''
    #Returns name and address of all files (including files in sub-durectory) containing a specific sub-string as dictionary.
    The file extentions are not included in the file name.

    #Parameters:
        address : string, address of the location of the files
        search_string: string, search object

    #Returns:
        dictioinary : A dictionary containing filenmae and address as key:value pair  
    '''
    file_address={}
    for root, dirs, files in os.walk(address): 
        for f in files:
            if search_string in f:
                file_address[f.replace(search_string,"")] = os.path.join(root, f) ## remove replace function to includer file extention
    return file_address

     


In [35]:
file_list = list_files_containing_search_string( search_string='.xlsx')    

In [38]:
master_list=[]


for file_name ,file_address in file_list.items():

    file= pd.read_excel(file_address, usecols=[0,1]) ## use only first two columns (# and Size)
    file.dropna(axis=0, inplace=True)
    file.columns =['organoid_id','size']

    row_id =list(file[file['organoid_id']=='#'].index) ## identify rows with '#' values, number of # values with 
    #consistent with number of wells 

    well_id=np.arange(0,len(row_id))

    row_id.append(file.index[-1]+1) ## need to append last-row-id+1 for better functionality of following processes

    for i in well_id:
        file.loc[row_id[i]+1:row_id[i+1]-1,'well_number'] = i+1 ## place well ids

    row_id.pop() # remove the value appended previosly 

    file=file.drop(row_id).reset_index().drop('index', axis=1).infer_objects()
    
    sample_name, sample_id = file_name.rsplit('_',1)
    file['sample_name'] = sample_name
    file['sample_id'] = sample_id

    file['size_cat']=np.where(file['size']<=50,'small',
                            np.where(file['size']<=150,'medium','large')) ##place categories
    
    master_list.append(file)

    save_address= file_address.replace('.xlsx','.csv') ## address to save the file

    file.to_csv(save_address)

master_file = pd.concat(master_list).reset_index().drop('index',axis=1)

master_file.to_csv('master_file_organoid_size.csv')



In [56]:
master_file = pd.read_csv('master_file_organoid_size.csv')

In [57]:
### groupby organoids using 'sample_name','sample_id','size_cat' and count them
mean_count_per_sample=master_file.groupby(['sample_name','sample_id','size_cat'],as_index=False)['organoid_id'].count().rename(columns={"organoid_id": "count"})


mean_count_per_sample.to_csv('mean_count_per_sample.csv')

In [58]:
count_file=master_file.groupby(['sample_name','sample_id','well_number','size_cat'],as_index=False)['organoid_id'].agg({'count':'count'})

count_file['count_mean']=count_file['count']/4  ##(devided by 4 because of 4 technical replicates)

In [59]:
count_file.to_csv('count_file_organoid_size.csv')

In [60]:
count_file.groupby(['sample_name','size_cat'],as_index=False).mean()

,sample_name,size_cat,well_number,count,count_mean
0,control,large,2.294118,47.529412,11.882353
1,control,medium,2.294118,188.647059,47.161765
2,control,small,2.294118,344.588235,86.147059
3,subject,large,2.500000,34.800000,8.700000
4,subject,medium,2.500000,211.600000,52.900000
5,subject,small,2.500000,397.550000,99.387500


In [61]:
count_file.groupby(['sample_name'],as_index=False).mean()

,sample_name,well_number,count,count_mean
0,control,2.294118,193.588235,48.397059
1,subject,2.500000,214.650000,53.662500


In [62]:
colony_efficiency=count_file.groupby(['sample_name','sample_id'],as_index=False)['count_mean'].agg({'count':'sum'})

# b['sample_id_mean_count']=b['count']/4

colony_efficiency['colony_forming efficiency']=(b['count']/8000)*100 ### started culture with 8000 cells

In [63]:
colony_efficiency.to_csv('colony_efficiency.csv')

In [64]:
colony_efficiency.groupby(['sample_name']).agg('mean','std')

,count,colony_forming efficiency
sample_name,,
control,493.65,6.170625
subject,643.95,8.049375
